In [1]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))  # GPU name

True
NVIDIA GeForce RTX 3090


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model


2025-10-05 20:33:40.541242: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-05 20:33:40.541267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-05 20:33:40.542166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-05 20:33:40.546311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-05 20:33:41.064593: W tensorflow/compiler/tf2

In [3]:
df = pd.read_csv("hf://datasets/SandeepKumarRudhravaram/Lung_Cancer_QA/Regenerated_Lung_Cancer_QA_Dataset.csv")
df

,input,output
0,What is the survival rate for lung cancer pati...,Numerous clinical trials are exploring targete...
1,How is lung cancer diagnosed?,"Air pollution, particularly fine particulate m..."
2,Is lung cancer hereditary?,"Air pollution, particularly fine particulate m..."
3,What are the stages of lung cancer?,Targeted therapies focus on specific genetic m...
4,Are there clinical trials for lung cancer trea...,Numerous clinical trials are exploring targete...
...,...,...
2995,What treatments are available for Stage 1 lung...,Numerous clinical trials are exploring targete...
2996,Are there alternative therapies for lung cancer?,The survival rate depends on the stage at diag...
2997,Are there clinical trials for lung cancer trea...,Numerous clinical trials are exploring targete...
2998,How is lung cancer diagnosed?,Some alternative therapies like acupuncture ma...


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Shape of training set: {train_df.shape}")
print(f"Shape of validation set: {validation_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of training set: (2400, 2)
Shape of validation set: (300, 2)
Shape of test set: (300, 2)


In [6]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [7]:
"""
# === Qwen3-32B 4-bit GPU setup and generation ===

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# --- Step 1: BitsAndBytes 4-bit config ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# --- Step 2: Load tokenizer ---
model_dir = "Qwen/Qwen3-32B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# --- Step 3: Load model on GPU(s) ---
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",       # Automatically distributes layers across GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    offload_folder="offload",  # Optional: offload layers to CPU if GPU memory is low
    offload_state_dict=True
)

# Disable caching to save GPU memory
model.config.use_cache = False
model.config.pretraining_tp = 1

# --- Step 4: Check GPU devices ---
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Model device map:", model.hf_device_map)

# --- Step 5: Generate text ---
prompt = "Once upon a time in a futuristic city,"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move inputs to GPU

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )

# Decode and print
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Generated Text ===")
print(generated_text)
"""

'\n# === Qwen3-32B 4-bit GPU setup and generation ===\n\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig\nimport torch\n\n# --- Step 1: BitsAndBytes 4-bit config ---\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=False,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=torch.bfloat16,\n)\n\n# --- Step 2: Load tokenizer ---\nmodel_dir = "Qwen/Qwen3-32B"\ntokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)\n\n# --- Step 3: Load model on GPU(s) ---\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_dir,\n    quantization_config=bnb_config,\n    device_map="auto",       # Automatically distributes layers across GPUs\n    torch_dtype=torch.bfloat16,\n    trust_remote_code=True,\n    offload_folder="offload",  # Optional: offload layers to CPU if GPU memory is low\n    offload_state_dict=True\n)\n\n# Disable caching to save GPU memory\nmodel.config.use_cache = False\nmodel.config.pretr

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# --- Step 1: 4-bit config for VRAM efficiency ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# --- Step 2: Load tokenizer ---
model_dir = "Qwen/Qwen2-0.5B"  # Choose the 0.5B model
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# --- Step 3: Load model across multiple GPUs ---
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",       # Automatically shard across all available GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Disable caching to save VRAM
model.config.use_cache = False

# --- Step 4: Determine first device for inputs ---
first_device = list(model.hf_device_map.values())[0]

# --- Step 5: Generate text ---
prompt = "What is lung cancer?"
inputs = tokenizer(prompt, return_tensors="pt").to(first_device)  # Place on first shard

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Generated Text ===")
print(generated_text)

# --- Step 6: Optional info ---
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Model device map:", model.hf_device_map)


`torch_dtype` is deprecated! Use `dtype` instead!
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


=== Generated Text ===
What is lung cancer? What are the main symptoms? What are the treatments for lung cancer? What can I do to prevent lung cancer from developing? How is lung cancer diagnosed? How is lung cancer treated? What are the different types of lung cancer?
Lung cancer is the most common type of cancer that affects the lungs. It is one of the largest cancer types in the United States. It is also the most common cancer in men and women.
Lung cancer is usually diagnosed by doctors using a combination of symptoms,
CUDA available: True
Number of GPUs: 2
Model device map: {'': 1}


In [9]:
def combine_qa(row):
    return f"Question: {row['input']}\nAnswer: {row['output']}\n"

train_texts = train_df.apply(combine_qa, axis=1).tolist()
val_texts = validation_df.apply(combine_qa, axis=1).tolist()


In [9]:
model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)



In [10]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",        # Multi-GPU
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False


In [11]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.input_ids[idx])
        }

train_dataset = QADataset(train_encodings)
val_dataset = QADataset(val_encodings)


In [12]:
"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically shard across GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False

"""

'\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=False,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=torch.bfloat16,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_name,\n    quantization_config=bnb_config,\n    device_map="auto",  # Automatically shard across GPUs\n    torch_dtype=torch.bfloat16,\n    trust_remote_code=True\n)\n\nmodel.config.use_cache = False\n\n'

In [13]:
device = torch.device("cuda:0")  # your GPU

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": device},  # pin model to single GPU
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False


In [14]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # adjust if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [15]:
training_args = TrainingArguments(
    output_dir="./qwen2-0.5B-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=4,
    save_total_limit=2
)


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


[codecarbon WARNING @ 14:13:16] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:13:16] [setup] RAM Tracking...
[codecarbon INFO @ 14:13:16] [setup] CPU Tracking...
[codecarbon WARNING @ 14:13:17] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 14:13:17] CPU Model on constant consumption mode: AMD Ryzen 7 5800X 8-Core Processor
[codecarbon WARNING @ 14:13:17] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 14:13:17] [setup] GPU Tracking...
[codecarbon INFO @ 14:13:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:13:17] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 14:

In [17]:
trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bobbyhieubui (bobbyhieubui-university-of-wisconsin-madison). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,6.968900
20,3.074200
30,0.229300
40,0.074000
50,0.056600
60,0.045000
70,0.034500
80,0.028300
90,0.023000
100,0.018700


[codecarbon INFO @ 14:13:38] Energy consumed for RAM : 0.000086 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 14:13:39] Delta energy consumed for CPU with cpu_load : 0.000045 kWh, power : 10.528193022900004 W
[codecarbon INFO @ 14:13:39] Energy consumed for All CPU : 0.000045 kWh
[codecarbon INFO @ 14:13:39] Energy consumed for all GPUs : 0.001260 kWh. Total GPU Power : 283.26316825020615 W
[codecarbon INFO @ 14:13:39] 0.001391 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:13:53] Energy consumed for RAM : 0.000167 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 14:13:54] Delta energy consumed for CPU with cpu_load : 0.000042 kWh, power : 10.5269794310625 W
[codecarbon INFO @ 14:13:54] Energy consumed for All CPU : 0.000088 kWh
[codecarbon INFO @ 14:13:54] Energy consumed for all GPUs : 0.002505 kWh. Total GPU Power : 298.87887095786846 W
[codecarbon INFO @ 14:13:54] 0.002759 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:14:08] Energy consumed for RAM :

TrainOutput(global_step=450, training_loss=0.23988944057789113, metrics={'train_runtime': 241.8257, 'train_samples_per_second': 29.774, 'train_steps_per_second': 1.861, 'total_flos': 7928092400025600.0, 'train_loss': 0.23988944057789113, 'epoch': 3.0})

In [19]:
model.save_pretrained("./qwen2-0.5B-lora-finetuned")
print("✅ Fine-tuning complete. LoRA weights saved.")


✅ Fine-tuning complete. LoRA weights saved.


In [22]:
import evaluate

# Load BLEU metric
bleu = evaluate.load("bleu")

predictions = []
references = []

for idx in range(len(val_dataset)):
    input_ids = val_dataset[idx]["input_ids"].unsqueeze(0).to(model.device)
    attention_mask = val_dataset[idx]["attention_mask"].unsqueeze(0).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512)

    pred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    ref_text = tokenizer.decode(val_dataset[idx]["labels"], skip_special_tokens=True)

    predictions.append(pred_text)       # <-- full string
    references.append([ref_text])       # <-- list of strings

# Compute BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU score:", bleu_score["bleu"])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `e

BLEU score: 0.9953218541612937
